# Getting Started with Strands Agents

This workshop introduces Strands Agents, a powerful framework for building AI agents with tool integration capabilities. You'll learn the fundamentals of creating agents, using built-in tools, and developing custom tools.

## Overview

In this lab, you will:
- Understand the core concepts of Strands Agents
- Create your first AI agent with built-in tools
- Build custom tools for specific use cases
- Explore conversation history and agent memory
- Learn best practices for agent development

## Prerequisites

Before starting this lab, ensure you have:
- AWS credentials configured (IAM role or environment variables)
- Required Python packages installed
- Nova Pro model ID based on AWS region

If you're not running in an environment with an IAM role assumed, set your AWS credentials as environment variables:

In [ ]:
import os

#os.environ["AWS_ACCESS_KEY_ID"]=<YOUR ACCESS KEY>
#os.environ["AWS_SECRET_ACCESS_KEY"]=<YOUR SECRET KEY>
#os.environ["AWS_SESSION_TOKEN"]=<OPTIONAL - YOUR SESSION TOKEN IF TEMP CREDENTIAL>
#os.environ["AWS_REGION"]=<AWS REGION WITH BEDROCK AGENTCORE AVAILABLE>

Install required packages for Strands Agents:

In [ ]:
#%pip install -q strands-agents strands-agents-tools rich

Setup Nova Pro model ID based on AWS region:

In [ ]:
import boto3

region = boto3.session.Session().region_name

NOVA_PRO_MODEL_ID = "us.amazon.nova-pro-v1:0"
if region.startswith("eu"):
    NOVA_PRO_MODEL_ID = "eu.amazon.nova-pro-v1:0"
elif region.startswith("ap"):
    NOVA_PRO_MODEL_ID = "apac.amazon.nova-pro-v1:0"

print(f"Nova Pro Model ID: {NOVA_PRO_MODEL_ID}")

## What are Strands Agents?

Strands Agents is a Python framework that simplifies the creation of AI agents with tool integration capabilities. Key features include:

- **Simple Agent Creation**: Easy-to-use API for creating AI agents with minimal code
- **Built-in Tools**: Pre-built tools like calculators, web search, and more
- **Custom Tool Support**: Create your own tools with simple Python functions
- **Conversation Memory**: Automatic conversation history management
- **Model Flexibility**: Support for various language models including models in Amazon Bedrock and OpenAI

Strands Agents provides a foundation for building sophisticated AI applications that can interact with external systems and perform complex tasks.

## Creating Your First Strands Agent

Let's start by creating a simple agent with a built-in calculator tool. This demonstrates the basic structure of Strands Agents and how tools are integrated:

In [ ]:
from strands import Agent, tool
from strands.models import BedrockModel
from strands_tools import calculator

# Create your first agent
agent = Agent(
    model=BedrockModel(model_id=NOVA_PRO_MODEL_ID),
    system_prompt="You are a helpful assistant that provides concise responses.",
    tools=[calculator],
)

agent("2+2=")

## Building Custom Tools

One of the powerful features of Strands Agents is the ability to create custom tools. Let's create a weather tool and combine it with the built-in calculator:

In [ ]:
from strands import Agent, tool
from strands.models import BedrockModel
from strands_tools import calculator

# Create a custom weather tool for demonstration
@tool
def weather(city: str) -> str:
    """Get weather information for a city
    Args:
        city: City or location name
    """
    return f"Weather for {city}: Sunny, 35°C" # dummy result for demo purpose

# Create your first agent
agent = Agent(
    model=BedrockModel(model_id=NOVA_PRO_MODEL_ID),
    system_prompt="You are a helpful assistant that provides concise responses.",
    tools=[weather, calculator],
)

print("\n=== Testing Agent Capabilities ===")
print("\n1. Testing custom weather tool:")
agent("How is the weather in HK?")

print("\n2. Testing calculator tool:")
agent("2+2=")

## Strands Agents Conversation Management

Strands Agents includes built-in conversation management with a default `SlidingWindowConversationManager` strategy. This system handles context management, memory optimization, and conversation flow to ensure agents can maintain coherent long-term interactions while respecting model context limits. This automatically maintains conversation context within sessions.

**Reference:** [Strands Agents Conversation Management](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/agents/conversation-management/)

In [ ]:
print("\n3. Testing conversation history capability in Strands Agents:")
agent("What did we talk about?")

## Understanding Agent Execution Flow

Let's examine how Strands Agents process requests and manage conversation history. This helps understand the internal workings of the agent framework:

In [ ]:
from rich.table import Table
import rich
import json

console = rich.get_console()

console.print("Agent Loop Detail")
console.rule()
console.print(f"Number of Loops: {agent.event_loop_metrics.cycle_count}")

table = Table(title="Agent Messages", show_lines=True)
table.add_column("Role", style="green")
table.add_column("Text", style="magenta")
table.add_column("Tool Name", style="cyan")
table.add_column("Tool Input", style="cyan")
table.add_column("Tool Result", style="cyan")

for message in agent.messages:
    text = [content["text"] for content in message["content"] if "text" in content]
    tool_name = [content["toolUse"]["name"] for content in message["content"] if "toolUse" in content]
    tool_input = [content["toolUse"]["input"] for content in message["content"] if "toolUse" in content]
    tool_result = [content["toolResult"]["content"][0] for content in message["content"] if "toolResult" in content]
    table.add_row(message["role"], text[-1] if text else "", 
                  tool_name[-1] if tool_name else "", 
                  json.dumps(tool_input[-1], indent=2) if tool_input else "", 
                  (json.dumps(tool_result[-1], indent=2)[:500]+"\n.\n.\n." if len(str(tool_result[-1])) > 500 else json.dumps(tool_result[-1], indent=2)) if tool_result else "")

console.print(table)

## Summary

In this introduction to Strands Agents, you've learned:

### What We Accomplished
- **Created your first AI agent** with built-in tools (calculator)
- **Built custom tools** using the `@tool` decorator for specific use cases
- **Explored conversation management** with different ConversationManager types
- **Examined agent execution flow** and message history tracking

### Additional Strands Capabilities
Beyond what we've covered, Strands Agents offers:

- **Model Context Protocol (MCP) Support**: Integrate with MCP servers for extended tool capabilities
- **Multi-Agent Patterns**: Coordinate multiple agents for complex workflows
- **Session Management**: Persistent conversation storage and retrieval
- **Streaming Responses**: Real-time response generation for better user experience
- **Custom Model Integration**: Support for various LLM providers beyond Amazon Bedrock

### Next Steps: Bedrock AgentCore Integration

Now that you understand the fundamentals of Strands Agents, we'll explore how to enhance these capabilities by integrating with **Amazon Bedrock AgentCore**. This integration provides:

- **Code Interpreter**: Execute Python code dynamically within agents
- **Browser Automation**: Web interaction and data extraction capabilities  
- **Secure Credential Management**: Safe handling of external API keys and secrets
- **Runtime Deployment**: Scalable agent deployment in cloud environments
- **Enhanced Observability**: Monitoring and debugging tools for production agents